In [1]:
from datetime import datetime
datetime.now().strftime("training_%Y-%m-%d.log")

'training_2025-04-01.log'

In [3]:
datetime.now().strftime("training_%Y-%m-%d_%H-%M-%S.log")

'training_2025-04-01_10-49-17.log'

In [4]:
8%4

0